# Pre-commit hooks
![pre-commit cover](./assets/pre-commit.jpg)

[Pre-commit](https://pre-commit.com/) hooks is a framework to manage your hooks that will run each time you will commit or push your code. (depending on how you set it)

The pre-commit hook itself will do nothing. It's just framework to automate commands and use utils.
We're gonna see some hanfy tools that we will couple with pre-commit hooks.

## Modules that we will use
### Black
[Black](https://pypi.org/project/black/) is a formatter for python. It will format all your code each time you commit/push following the PEP8 standart.

### MyPy
[MyPy](http://mypy-lang.org/) will check all your code to see if there is any typing issues.

### Flake8
[Flake8](https://medium.com/python-pandemonium/what-is-flake8-and-why-we-should-use-it-b89bd78073f2) will check if you forgot to add  docstring to your functions  and classes, if you use the correct cases in your variables/functions/classes, if you forgot to import a module that you're using, if there is unsed import,...
Flake8  is in fact a wrapper of these tools in one command:
* PyFlakes
* pycodestyle
* Ned Batchelder’s McCabe script

### Isort
[Isort](https://pypi.org/project/isort/) is an utils that will sort your imports in a way that make sense and  that is optimized.

### And so much more
But that's not all! You can find a lot of other modules or even create your own!
You can for example add a step to check if there is no files bigger than X or if the branch name respect a convention,...


## Why using these tools?
It will  make you gain a lot of time during PR and I recomnand you to run it before launching all your tests. It will avoid you to run test that take a while to run to discover, during the last test that you forgot an import in a file. Or simply improve  you code's speed because you're sure that there is no module imported when you don't use it. The formating will also improve a lot the readibilty of your code and that's something that you can totally forget to focus on the most import thing: fixing your business problem and not check if you haven't forgot to type all the functions or to add  a space after a parenthesis. 

## How does it works?
1. **Installation and dependencies**
First, we will need to install pre-commit from the terminal. You can use pip to do it.

```bash
pip install pre-commit
```

You will also need to install all the modules that we want to use. 

```bash
pip install black mypy isort flake8
```

2. **Configuration**
Now we will define the behavior of pre-commit. To do so, we need to create a yaml file and specify what do we expect.
The file should be nammed `.pre-commit-config.yaml`. A basic configuration for what we want will be:

```yaml
repos:
-   repo: local
    hooks:
    - id: isort
      name: isort
      description: 'Sort imports'
      entry: isort
      language: system
      types: [python]
      # We defined that we want to run this step when we try to commit.
      # If you want to apply it before push juste replace commit with push.
      stages: [commit]
      # Add all the arguments you want to the Isort command here
      # Make sure to make it compatible with black
      args: 
        - -rc
        - --lines=120
        - --use-parentheses
        - --multi-line=3
    - id: black
      name: black
      language: system
      description: 'Format code'
      entry: black
      types: [python]
      args:
        - --line-length=122
    - id: flake8
      name: flake8
      description: 'Check logic issues'
      language: system
      entry: flake8
      types: [python]
 
    - id: mypy
      name: mypy
      description: 'Check typing'
      language: system
      entry: mypy
      types: [python]
```

## Demo
Everything is ready, let's see how does it works now!

To demonstrate how does it works, I prepared some files that contain some errors and we will see how to fix them.

**WARNING:** As these modules are gonna fix the issues, you will not be able to re-run this notebook twice and see the errors again.

### Black
![black logo](./assets/black.jpg)
The file `bad_fomrating.py` was wrote by a programmer that was super hungry. So he didn't took the time to properly format his code. Let's have a look:

In [12]:
!cat code/bad_formating.py

# Damn, I am super hungry. Let's rush that code asap!
def   addition    (a=1,b=3):
  result=a+b
  return f"result is "    +str(result )

print(addition(1,2))


Uggly right? Even if it's really badly formatted, the code run:

In [8]:
!python3 code/bad_formating.py

result is 3


If you're curious and you want to see how black will reformat a file before writing your can do:

*(it will only show you what it plan to do without wrinting on the file)*

In [17]:
!black --color --diff code/bad_formating.py

would reformat code/bad_formating.py
All done! ✨ 🍰 ✨
1 file would be reformatted.
--- code/bad_formating.py	2020-08-27 13:55:57.241139 +0000
+++ code/bad_formating.py	2020-08-27 13:57:39.563537 +0000
@@ -1,6 +1,7 @@
 # Damn, I am super hungry. Let's rush that code asap!
-def   addition    (a=1,b=3):
-  result=a+b
-  return f"result is "    +str(result )
+def addition(a=1, b=3):
+    result = a + b
+    return f"result is " + str(result)
 
-print(addition(1,2))
+
+print(addition(1, 2))


Let's fix that before our eyes burn. We will use direcly black in the terminal.
Now that black is installed, you can simply use the `black` command followed by the path of the file/folder that you want to check. When we will  run the pre-commit hook it will run `black .` to formatted everything that is inside the current folder.

In [13]:
!black code/bad_formating.py

reformatted code/bad_formating.py
All done! ✨ 🍰 ✨
1 file reformatted.


Black reformatted the file. Let's see how  did it saved the day:

In [14]:
!cat ./code/bad_formating.py

# Damn, I am super hungry. Let's rush that code asap!
def addition(a=1, b=3):
    result = a + b
    return f"result is " + str(result)


print(addition(1, 2))


As we can see, everything is well formatted now.

### MyPy
![mypy logo](./assets/mypy.png)
The file `missing_annotation.py` was wrote by the new intern of the company. He **ALWAYS** forget to add typing on **EACH** pull request.
You're more then tired to copy paste "Please add typing here.". You also see that he often forget to add returns in case these confitions aren't true. Let's check this guy's code:

In [18]:
!cat ./code/missing_annotation.py

def divide(a: int,  b) -> int:
    if b != 0:
        return a / b

To help him to fix these issues without needing to use more your already too used C and  V keys, you make him run mypy:

In [20]:
!mypy --strict ./code/missing_annotation.py

code/missing_annotation.py:1: error: Function is missing a type annotation for one or more arguments
code/missing_annotation.py:1: error: Missing return statement
code/missing_annotation.py:3: error: Returning Any from function declared to return "int"
Found 3 errors in 1 file (checked 1 source file)


Thanks to you he's now able to fix his code by himself. Let's see the improvements:

In [23]:
!cat ./code/fixed_annotation.py

from typing import Union

def divide(a: int,  b: int) -> Union[float, str]:
    if b != 0:
        return float(a / b)
    else:
        return "Can't divide by zero. Please change the value of b."

Look better right? Let's see if there is still any issues with this code.

In [24]:
!mypy --strict ./code/fixed_annotation.py

Success: no issues found in 1 source file


Perfect, you just won hours of PR review you can now focus on you own work!

## Flake8

You finnaly take a well deserved weak of vacation on a beautiful beach. When you comeback at work, you PM tell you that during your rest they refactor all the codebase of you project. You suspiciously run you performents test and you find that the code is way slower than it should. You  also see that a lot of you unit-test doesn't run anymore.

You just came back and you don't want to spend hours an code review. So the first thing you will do is runnng flake8 during this time you grab a cofee, ready to fight this code! It won't fix the issue  for you but at least you know where to start.

Let's have  a look at the code:

In [27]:
!cat ./code/refactored_code.py

"""Hey Jonh, I hope you enjoyed your vacations. There is some issues with this code. Welcome back!"""
import subprocess

def divide(a: int,  b: int) -> Union[float, str]:
    if b != 0:
        return float(a / b)
    else:
        return "Can't divide by zero. Please change the value of b."


def addition(a:  int, b: int) -> int:
    result = a + b
    return f"result is {str(result)}"

def just_another_function(a: int) -> float:
    return a * math.pi

Let's see how much issues MyPy can find!

In [28]:
!mypy ./code/refactored_code.py

code/refactored_code.py:4: error: Name 'Union' is not defined
code/refactored_code.py:4: note: Did you forget to import it from "typing"? (Suggestion: "from typing import Union")
code/refactored_code.py:13: error: Incompatible return value type (got "str", expected "int")
code/refactored_code.py:16: error: Name 'math' is not defined
Found 3 errors in 1 file (checked 1 source file)


You cofee is ready, you didn't touched a keyboard for more than 1 week now, you fix the code is easly thanks to mypy.

In [29]:
!cat ./code/refactored_refactored_code.py

"""Hey Jonh, I hope you enjoyed your vacations. There is some issues with this code. Welcome back!"""
from typing import Union
import math

def divide(a: int,  b: int) -> Union[float, str]:
    if b != 0:
        return float(a / b)
    else:
        return "Can't divide by zero. Please change the value of b."


def addition(a:  int, b: int) -> str:
    result = a + b
    return f"result is {str(result)}"

def just_another_function(a: int) -> float:
    return a * math.pi

In [30]:
!mypy ./code/refactored_refactored_code.py

Success: no issues found in 1 source file


Nohing to add, you're simply the best. You haven't finished you cup of coffee that you're code is running again.

##  Isort
![isort logo](./assets/isort.jpg)

You work on a project that have a lot of imports and it's not really lisible. When you have to find a function that is imported from your own file it's hard to find... Let's order that!

In [36]:
!cat ./code/unsorted_import.py

import math
from typing import Union
import subprocess
from refactored_refactored_code import addition
import statistics
from typing import Dict
import time
import date
from code.fixed_annotation import divide

In [37]:
!isort ./code/unsorted_import.py

Fixing /mnt/c/Users/maxim/code/becode/Python-Upskilling/I-O/02.Pre_commit_hook/code/unsorted_import.py


In [38]:
!cat ./code/unsorted_import.py

import math
import statistics
import subprocess
import time
from code.fixed_annotation import divide
from typing import Dict, Union

import date
from refactored_refactored_code import addition


## All in one
Perfect! but it's a lot to do mannualy, isn't it? That's where pre-commit hooks are gonna save us a **lot** of time!

We defined that we wanted to use all the precedents tools and  how in our `.pre-commit-config.yaml`. So you juste have to run the command: `pre-commit run --all` in order to run all those tools on the complete current folder. or `pre-commit run --files <your_files>` to only run it on certain files.

In [47]:
!pre-commit run --files ./code/fixed_annotation.py

isort....................................................................Passed
black....................................................................Passed
flake8...................................................................Passed
mypy.....................................................................Passed


Everything is fine, the commit or the push can be done safely. Let's see if something is wrong:

In [48]:
!pre-commit run --files ./code/refactored_code.py

isort....................................................................Passed
black....................................................................Passed
flake8...................................................................Failed
- hook id: flake8
- exit code: 1

I-O/02.Pre_commit_hook/code/refactored_code.py:1:80: E501 line too long (101 > 79 characters)
I-O/02.Pre_commit_hook/code/refactored_code.py:2:1: F401 'subprocess' imported but unused
I-O/02.Pre_commit_hook/code/refactored_code.py:5:31: F821 undefined name 'Union'
I-O/02.Pre_commit_hook/code/refactored_code.py:18:16: F821 undefined name 'math'

mypy.....................................................................Failed
- hook id: mypy
- exit code: 1

I-O/02.Pre_commit_hook/code/refactored_code.py:5: error: Name 'Union' is not defined
I-O/02.Pre_commit_hook/code/refactored_code.py:5: note: Did you forget to import it from "typing"? (Suggestion: "from typing import Union")
I-O/02.Pre_commit_hook/code/refactored_cod

## Automatise the process
If we don't won't to run it each time before commit we can simple install it and it wil run on each commit or push depending on the config. If everything pass, the commit/push will be done. If there is issues, it will be canceled. You can of course change everything depending on your needs. Only apply it on merge, add or remove modules,...

## You turn!
Alright, you have all the keys to have a cleaner code and to lose less time!

![gif about time](./assets/time.gif "segment")